<a href="https://colab.research.google.com/github/nancymatijas/OSiRV-Projekt/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip3 install med2image

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for med2image: filename=med2image-2.6.6-py3-none-any.whl size=18863 sha256=ea52237f29c7675c6833095cf37866e14b90a5c477127ee2f6ae06955fae227d
  Stored in directory: /root/.cache/pip/wheels/07/cc/8e/2f95f14d9c19410733709350972e7e8dc

# **PRETVORBA IZ .NII.GZ U .JPG**

# *Svaki .nii.gz slice u 154 slike*

In [ ]:
import os

# Path to the main directory
main_directory = '/content/drive/MyDrive/MICCAI_BraTS2020_TrainingData'

# Create the output directory for converted images
output_directory = '/content/drive/MyDrive/output_images'

# Create the main output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# List all subdirectories in the main directory
subdirectories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

# Loop through each subdirectory and convert files
for subdirectory in subdirectories:
    input_directory = os.path.join(main_directory, subdirectory)

    # Create the output directory within output_images
    output_case_directory = os.path.join(output_directory, subdirectory)
    os.makedirs(output_case_directory, exist_ok=True)

    # List all files in the subdirectory
    files = [f for f in os.listdir(input_directory) if f.endswith('.nii.gz')]

    # Loop through each file and convert
    for file in files:
        input_file = os.path.join(input_directory, file)
        output_file_stem = f"{subdirectory}_{os.path.splitext(file)[0]}"

        # Modify other parameters as needed
        command = f"med2image -i '{input_file}' -d '{output_case_directory}' -o '{output_file_stem}' --outputFileType jpg"

        try:
            # Execute the command
            os.system(command)
            print(f"Conversion successful for {input_file}")
        except Exception as e:
            print(f"Error converting {input_file}: {e}")

print("Conversion completed for all directories.")

# *Spajanje 154 slice-a za svaki presjek u 1 slice*

In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image

# Path to the main directory
main_directory = '/content/drive/MyDrive/output_images'

# Output directory for combined images
combined_output_directory = '/content/drive/MyDrive/MICCAI_BraTS2020_TrainingData_comb'

# Create the main combined output directory if it doesn't exist
os.makedirs(combined_output_directory, exist_ok=True)

# List all subdirectories in the main directory
subdirectories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

# Loop through each subdirectory and combine files
for subdirectory in subdirectories:
    input_directory = os.path.join(main_directory, subdirectory)

    # Create the output directory within combined_output_directory
    combined_case_directory = os.path.join(combined_output_directory, subdirectory)
    os.makedirs(combined_case_directory, exist_ok=True)

    # Initialize empty arrays for each image type
    t1_array = []
    t2_array = []
    flair_array = []
    seg_array = []
    t1ce_array = []

    # List all files in the subdirectory
    files = [f for f in os.listdir(input_directory) if f.endswith('.jpg')]

    for file in files:
      input_file = os.path.join(input_directory, file)
      #print(f"Processing file: {file}")

      # Load the image using an image processing library (e.g., PIL, OpenCV)
      img = Image.open(input_file)
      img_array = np.array(img)

      # Assuming that each image corresponds to a different slice, you can concatenate them along the appropriate axis
      if 't1' in file.lower():
          t1_array.append(img_array)
      elif 't2' in file.lower():
          t2_array.append(img_array)
      elif 'flair' in file.lower():
          flair_array.append(img_array)
      elif 'seg' in file.lower():
          seg_array.append(img_array)
      if 't1ce' in file.lower():
          t1ce_array.append(img_array)

    # Stack images along a new axis (axis=3) to create a 4D volume
    if t1_array:
        t1_combined = np.stack(t1_array, axis=3)
        Image.fromarray(np.uint8(np.mean(t1_combined, axis=3))).save(os.path.join(combined_case_directory, f'{subdirectory}_T1_combined.jpg'))
        print(f"Combined T1 images for {subdirectory}")
    if t2_array:
        t2_combined = np.stack(t2_array, axis=3)
        Image.fromarray(np.uint8(np.mean(t2_combined, axis=3))).save(os.path.join(combined_case_directory, f'{subdirectory}_T2_combined.jpg'))
        print(f"Combined T2 images for {subdirectory}")
    if flair_array:
        flair_combined = np.stack(flair_array, axis=3)
        Image.fromarray(np.uint8(np.mean(flair_combined, axis=3))).save(os.path.join(combined_case_directory, f'{subdirectory}_FLAIR_combined.jpg'))
        print(f"Combined FLAIR images for {subdirectory}")
    if seg_array:
        seg_combined = np.stack(seg_array, axis=3)
        Image.fromarray(np.uint8(np.mean(seg_combined, axis=3))).save(os.path.join(combined_case_directory, f'{subdirectory}_SEG_combined.jpg'))
        print(f"Combined SEG images for {subdirectory}")
    if t1ce_array:
        t1ce_combined = np.stack(t1ce_array, axis=3)
        Image.fromarray(np.uint8(np.mean(t1ce_combined, axis=3))).save(os.path.join(combined_case_directory, f'{subdirectory}_T1CE_combined.jpg'))
        print(f"Combined T1CE images for {subdirectory}")


print("Combination completed for all directories.")

# *Po jedna slika za svaki slice*

In [ ]:
import os

# Path to the main directory
main_directory = '/content/drive/MyDrive/MICCAI_BraTS2020_TrainingData'

# Create the output directory for converted images
output_directory = '/content/drive/MyDrive/MICCAI_BraTS2020_TrainingData_jpg'

# Create the main output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# List all subdirectories in the main directory
subdirectories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

# Loop through each subdirectory and convert files
for subdirectory in subdirectories:
    input_directory = os.path.join(main_directory, subdirectory)

    # Create the output directory within output_images
    output_case_directory = os.path.join(output_directory, subdirectory)
    os.makedirs(output_case_directory, exist_ok=True)

    # List all files in the subdirectory
    files = [f for f in os.listdir(input_directory) if f.endswith('.nii.gz')]

    # Loop through each file and convert
    for file in files:
        input_file = os.path.join(input_directory, file)
        # Extract file name without extension
        file_name_without_extension = os.path.splitext(file)[0]

        # Remove the redundant part from the file name
        if file_name_without_extension.startswith(subdirectory):
            file_name_without_extension = file_name_without_extension[len(subdirectory) + 1:]

        # Combine subdirectory name and modified file name without extension
        output_file_stem = f"{subdirectory}_{file_name_without_extension}"

        # Modify other parameters as needed, adding --sliceToConvert
        command = f"med2image -i '{input_file}' -d '{output_case_directory}' -o '{output_file_stem}' --outputFileType jpg --sliceToConvert m"

        try:
            # Execute the command
            os.system(command)
            print(f"Conversion successful for {input_file}")
        except Exception as e:
            print(f"Error converting {input_file}: {e}")


print("Conversion completed for all directories.")

# **SORTIRANJE PO PRESJECIMA**

In [ ]:
import shutil

# Path to the main directory
main_directory = '/content/drive/MyDrive/MICCAI_BraTS2020_TrainingData_jpg'

# Create the output directory for images
output_directory = '/content/drive/MyDrive/MICCAI_BraTS2020_TrainingData_sorted'
os.makedirs(output_directory, exist_ok=True)

# List all subdirectories in the main directory
subdirectories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

# Loop through each subdirectory
for subdirectory in subdirectories:
    input_subdirectory = os.path.join(main_directory, subdirectory)

    # Create subdirectories 't1', 't2', 't1ce', 'flair', 'seg' within output2 for each subdirectory
    for image_type in ['t1', 't2', 't1ce', 'flair', 'seg']:
        type_directory = os.path.join(output_directory, image_type)
        os.makedirs(type_directory, exist_ok=True)

    # List all files in the subdirectory
    files = [f for f in os.listdir(input_subdirectory) if f.endswith('.jpg')]

    # Loop through each file and copy to the corresponding type directory
    for file in files:
        input_file = os.path.join(input_subdirectory, file)

        # Extract type from the file name
        image_type = file.split('_')[-1].split('-')[0]

        # Destination directory
        output_type_directory = os.path.join(output_directory, image_type)
        output_file = os.path.join(output_type_directory, file)
        shutil.copy(input_file, output_file)
        print(f"Copying {file} to {output_type_directory}")

print("Copying completed for all JPG images.")